Code for Comparisons of the Deep Learning and Traditional Compression Methods

In [1]:
#importing the modules for the SIFT features part
import cv2
import os
import numpy as np
import time
from numpy import polyfit
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from PIL import Image

In [2]:
#importing modules for the Compression Part

import math
import torch
from torchvision import transforms
from torchvision.utils import save_image

from pytorch_msssim import ms_ssim #metric for the image
import pandas as pd
import pillow_avif #AVIF encoder plugin for the Pillow installed using, pip install pillow-avif-plugin
import lpips #already installed with pip install lpips 
#used for the lpips metric
loss_fn_alex=lpips.LPIPS(net="alex")
loss_fn_vgg=lpips.LPIPS(net="vgg") 

#another image reconstruction quality metric
from skimage.metrics import structural_similarity as ssim 
#PSNR metric for image reconstruction
from skimage.metrics import peak_signal_noise_ratio as psnr

from compressai.zoo import bmshj2018_factorized #VAE model for image compression
from ipywidgets import interact, widgets

Setting up [LPIPS] perceptual loss: trunk [alex], v[0.1], spatial [off]


C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\lpips\weights\v0.1\alex.pth
Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: C:\Swapnil\Narrowband_DRONE\Image_compression_code\Method_7_compress_ai\compressAIenv\Lib\site-packages\lpips\weights\v0.1\vgg.pth


In [3]:
#setting the CUDA device
device="cuda" if torch.cuda.is_available() else "cpu"
print("Using:",device)

Using: cuda


Deep Learning Model for Image compression

In [36]:

def DL_model(metric="ms-ssim",quality=6):
    """
    Using BMSHJ2018_factorized version.
    """
    net=bmshj2018_factorized(quality=quality,metric=metric,pretrained=True).eval().to(device)
    print(f"Parameters:{sum(p.numel() for p in net.parameters())}")
    return net
    

quality=8
metric="ms-ssim"
net=DL_model(metric,quality)

Parameters:7030531


Deep Learning Model based Image dataset compression

In [5]:
#Stage 1 DL Compression 
def compress_dataset_DL(model,images,dataset_path,img_type='RGB',export_format='webp'):
    '''
    img_type argument expects 'RGB' for color image and 'L' for grayscale
    USES save_image method which does not have controls
    '''
    img_details={} #dictionary to store the details of the images

    if export_format=='webp':
        extension='.webp'
    if export_format=='avif':
        extension='.avif'
    #make the export directory
    try:
        directory_name="Data_"+export_format
        os.mkdir(directory_name)
    except:
        print("Directory already exists")
    print(dataset_path)
    export_path=os.path.join(dataset_path,directory_name)

    compression_time=[]
    
    for img in images:
        start=time.time()
        print("Doing for:",img)
        img_path=os.path.join(dataset_path,img)
        #print("img_path:",img_path)
        #open the img
        image=Image.open(img_path).convert(img_type) #opening the image as RGB
        x=transforms.ToTensor()(image).unsqueeze(0).to(device) #convert to PyTorch tensor and send to device
        
        with torch.no_grad():
            out_net=net.forward(x)
        out_net['x_hat'].clamp(0,1)
        obj=out_net['x_hat']

        #save the output of the network as image
        #saving in the tensor form takes lot of space that is why
        #directly exporting the tensor output as image (94 MB vs 1MB)
        #print(img)
        img=img.split('.')[0] + extension #getting the image name and changing the extension
        img=os.path.join(export_path,img)
        save_image(obj,img)
        end=time.time()
        compression_time.append(end-start)
    return export_path,compression_time

In [6]:
#Stage 2 compression 
#OverWriting the stage 1 compression results from stage 2
def compress_stage_2(img_org_path,img_recon_path,export_format="avif",quality=10):
    """ 
    Input: Path to already compressed images from method save_image
    Output: OverWrite the existing images
    Use Pillow here to export the images

    export_format="avif","webp"
    quality=0 to 100
    
    """
    
    img_details={} #dictionary to store the details of the images

    if export_format=='webp':
        extension='.webp'
    else:
        extension='.avif'

    images,path=file_handling(img_recon_path)
    compression_time=[]
    for img in images:
        start=time.time()
        print("Doing for:",img)
        img_path=os.path.join(path,img)
        image_name=img.split('.')[0] + "_c"+ extension #getting the image name and changing the extension
        with Image.open(img_path) as image:
            image.save(image_name,format=export_format.upper(),quality=quality)
        #remove previous file
        os.remove(img)
        end=time.time()
        compression_time.append(end-start)
    return compression_time


In [7]:
#DL COMPRESSION COMPLETE FUNCTION

def DL_compression(model,images,dataset_path):
    export_path,compression_time=compress_dataset_DL(model,images,dataset_path,img_type='RGB',export_format='webp')
    #compression_time is the list of time spend on Stage 1 (DL) compression of the image
    img_org_path=dataset_path
    img_recon_path=export_path
    
    compression_time_2=compress_stage_2(img_org_path,img_recon_path,export_format="webp",quality=5)
    #compression_time_2 is the list of time spent on compression of each image in stage 2
    
    #total_time is the total time on the compression of each image in both the stage 
    total_time=[i+j for i,j in zip(compression_time,compression_time_2)]
    return total_time

Traditional Method based Image Compression

In [45]:

def create_empty_dir(path,dir_name):
    export_path=os.path.join(path,dir_name)
    try:
        os.mkdir(export_path)
    except:
        print("Directory already exists")
        #removing all the files
        files=os.listdir(export_path)
        for file in files:
            os.remove(os.path.join(export_path,file))
        print("Removed all the files in directory")
    print("\n Directory created \n")
    return export_path

def compress_dataset_trad(dataset_path,quality=10):
    """
    Here we directly compress the images using the traditional codecs like JPEG WEBP AVIF

    Challenge is to What compression ratio it must be compressed.
    Rather for the time being we are using quality parameter of 10 to achieve very high 
    compression at the cost of reduced quality in the compression process of the image.
    """
    images,path=file_handling(dataset_path) #get the list of images and the path to the dataset

    #JPEG

    export_path=create_empty_dir(path,"JPG_C")
        

    for image in images:
        img=Image.open(os.path.join(path,image))
        export_name=os.path.join(export_path,image.split('.')[0]+"_c.jpg")
        img.save(export_name,format="JPEG",quality=quality)

    #WEBP
    export_path=create_empty_dir(path,"WEBP_C")
        

    for image in images:
        img=Image.open(os.path.join(path,image))
        export_name=os.path.join(export_path,image.split('.')[0]+"_c.webp")
        img.save(export_name,format="WEBP",quality=quality)
        
    #AVIF

    export_path=create_empty_dir(path,"AVIF_C")
        

    for image in images:
        img=Image.open(os.path.join(path,image))
        export_name=os.path.join(export_path,image.split('.')[0]+"_c.avif")
        img.save(export_name,format="AVIF",quality=quality)

    
    
    
    

RUNNING THE COMPRESSION PROCESS

In [14]:
dataset_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images"

#Running the Traditional Process
compress_dataset_trad(dataset_path)



NameError: name 'compress_dataset_trad' is not defined

In [38]:
#Running the DL Compression

images,path=file_handling(dataset_path)
total_time=DL_compression(net,images,dataset_path)


Images in dataset: 50
First filename: 000000_left.png
Directory already exists
C:\Swapnil\Narrowband_DRONE\compression_experiment\images
Doing for: 000000_left.png
Doing for: 000001_left.png
Doing for: 000002_left.png
Doing for: 000003_left.png
Doing for: 000004_left.png
Doing for: 000005_left.png
Doing for: 000006_left.png
Doing for: 000007_left.png
Doing for: 000008_left.png
Doing for: 000009_left.png
Doing for: 000010_left.png
Doing for: 000011_left.png
Doing for: 000012_left.png
Doing for: 000013_left.png
Doing for: 000014_left.png
Doing for: 000015_left.png
Doing for: 000016_left.png
Doing for: 000017_left.png
Doing for: 000018_left.png
Doing for: 000019_left.png
Doing for: 000020_left.png
Doing for: 000021_left.png
Doing for: 000022_left.png
Doing for: 000023_left.png
Doing for: 000024_left.png
Doing for: 000025_left.png
Doing for: 000026_left.png
Doing for: 000027_left.png
Doing for: 000028_left.png
Doing for: 000029_left.png
Doing for: 000030_left.png
Doing for: 000031_left.png

Utilities for the Compression and Analysis

#FILE HANDLING

In [13]:
def file_handling(path):
    """
    path to the images dataset
    """
    #org_dir=os.getcwd()

    #changing to the dataset dir
    try:
        os.chdir(path)
    except:
        print("Path not valid")
        return

    files=os.listdir(path) #files in the directory
    images=[] #getting only the jpg and png images
    for file in files:
        if file.lower().endswith(".jpeg") or file.lower().endswith(".jpg") \
        or file.lower().endswith(".png") or file.lower().endswith(".webp") \
        or file.lower().endswith(".avif"):
            images.append(file)

    print("Images in dataset:",len(images))
    if len(images)!=0:
        print("First filename:",images[0])
    else:
        print("No images in the path or different format")
    
    #go to the original directory back
    #os.chdir(org_dir)
    
    return images, path

#Metrics for calculation of compression quality

In [17]:
def crop_image(img_org_path,img_recon_path):
    """
        top left X and Y
        bottom right X and Y is the crop box
    """

    img_recon=Image.open(img_recon_path)
    img_org=Image.open(img_org_path)

    output_width, output_height =img_recon.size
    original_width,original_height=img_org.size

    #checking the dimensions of the images
    if original_width==output_width and \
    original_height==output_height:
        return None
    else:
        crop_box = (0, 0, output_width, original_height)
        cropped_image = img_recon.crop(crop_box)
        #print(cropped_image.size)
        return cropped_image

def calculate_ssim(img_org,img_recon):
    """
    img_org and img_recon are the path of the 
    original image and the reconstructed image
    """
    if crop_image(img_org,img_recon) is None:
        img_recon=np.asarray(Image.open(img_recon))
        img_org=np.asarray(Image.open(img_org))
    else:
        img_recon=np.asarray(crop_image(img_org,img_recon))
        img_org=np.asarray(Image.open(img_org))
        
    ssim_value=ssim(img_org,img_recon,win_size=3) #for 3 channel
    #print("SSIM:",ssim_value)
    return ssim_value

def calculate_psnr(img_org,img_recon):

    """
    img_org and img_recon are the path of the 
    original image and the reconstructed image
    """
    
    if crop_image(img_org,img_recon) is None:
        img_recon=np.asarray(Image.open(img_recon))
        img_org=np.asarray(Image.open(img_org))
    else:
        img_recon=np.asarray(crop_image(img_org,img_recon))
        img_org=np.asarray(Image.open(img_org))

    psnr_value=psnr(img_org,img_recon)
    #print("PSNR:",psnr_value)
    return psnr_value


def calculate_lpips(img_org,img_recon):
    """
    img_org and img_recon are the path of the 
    original image and the reconstructed image
    """
    if crop_image(img_org,img_recon) is None:
        img_recon=np.asarray(Image.open(img_recon))
        img_org=np.asarray(Image.open(img_org))
    else:
        img_recon=np.asarray(crop_image(img_org,img_recon))
        img_org=np.asarray(Image.open(img_org))
    
    transform=transforms.ToTensor() #convert the image to Tensor 
    #print(img_org.shape,img_recon.shape)
    img_org=transform(img_org)
    img_recon=transform(img_recon)
    
    lpips=loss_fn_alex.forward(img_org,img_recon)
    lpips=lpips.detach().numpy() #to cpu then 
    #print("LPIPS:",lpips) 
    return lpips[0][0][0][0]
    

def compression_ratio(img_1_path,img_2_path):
    """
        Assuming the first image is uncompressed version
    """
    print(img_1_path)
    try:
        size_1=os.path.getsize(img_1_path) #getting the size of the image on the disc
    except:
        print("img_1 path not valid")

    try:
        size_2=os.path.getsize(img_2_path) #getting the size of the image on the disc
    except:
        print("img_2 path not valid")

    ratio=size_1/size_2
    #print("Compression ratio is:",ratio)
    return ratio

FEATURES IN IMAGES

In [18]:
#SIFT FEATURES

def sift(image):
    '''
    Function can draw keypoints on the image too, 
    but calculation is commented.
    '''

    if image.endswith(".avif"):
        image=Image.open(image).convert("L")
        image=np.array(image)
    else:       
        image=cv2.imread(image,cv2.IMREAD_GRAYSCALE)
    
    sift=cv2.SIFT.create() #initiate SIFT detector

    #Find keypoints on the image
    keypoints,descriptors=sift.detectAndCompute(image,None)

    #Draw keypoints on the image
    #image_with_sift=cv2.drawKeyPoints(image.copy(),keypoints,None,(255,0,0),4)

    return keypoints#,image_with_sift

In [34]:
#Utility function for writing the dictionary to the file

def write_dct_file(dct,filename):
    with open(filename,'w') as f:
        for metric, value in dct.items():
            f.write(f"{metric}:{value}\n")

Analysis for Dataset

In [35]:
#calculate the metrics for the batch
def batch_metrics(org_path,recon_path):
    """
    Input: Path of the original dataset
    Output: Dataframe of reconstruction quality
            metrics and compression ratio for each
            image in dataset
    """
    
    images,path=file_handling(org_path)
    images_r,path_r=file_handling(recon_path)
        

    #initialize the metrics
    ssim=[]
    psnr=[]
    lpips=[]
    cr=[]
    org_size=[]
    recon_size=[]
    sift_features_o=[]
    sift_features_r=[]
    sift_features_per_red=[]

    for img_o,img_r in zip(images,images_r):

        print("Doing for:",img_o)

        img_o=os.path.join(path,img_o)
        img_r=os.path.join(path_r,img_r)
        
        ratio=compression_ratio(img_o,img_r)
        cr.append(round(ratio,2))
        
        lpips_=calculate_lpips(img_o,img_r)
        lpips.append(lpips_)
    
    
        ssim_=calculate_ssim(img_o,img_r)
        ssim.append(ssim_)

        
        psnr_=calculate_psnr(img_o,img_r)
        psnr.append(psnr_)

        #sizes are in KB
        org_size.append(os.path.getsize(os.path.join(path,img_o))/1000)
        recon_size.append(os.path.getsize(os.path.join(path_r,img_r))/1000)

        features_o=len(sift(img_o)) #returns the list , we need count of those features
        sift_features_o.append(features_o)

        features_r=len(sift(img_r)) #returns the list , we need count of those features
        sift_features_r.append(features_r)

        sift_features_per_red_=(features_o-features_r)/(features_o)*100
        sift_features_per_red.append(sift_features_per_red_)
        
        

        
        

    metrics={"SSIM":ssim,
             "PSNR":psnr,
             "LPIPS":lpips,
             "CR":cr,
            "ORG_SIZE":org_size,
            "RECON_SIZE":recon_size,
            "ORG_SIFT":sift_features_o,
            "RECON_SIFT":sift_features_r,
            "SIFT_PER_RED":sift_features_per_red}

    #make a dataframe from the metrics for exporting
    df=pd.DataFrame(metrics)
    return df

def average_batch_metrics(df,filename):
    """
    Gives the average of each of the metric 
    for the whole batch
    """
    cols=df.columns
    avg_metric={}

    #number of images
    avg_metric["Dataset_size"]=len(df[cols[0]])
    for metric in cols:
        metric_array=np.array(df[metric]) #convert the each column of df to np array
        avg_metric[metric]=round(np.average(metric_array),2)
    
    write_dct_file(avg_metric,filename)
    print("\n Exported the text form of the summary\n")
    print("average_metrics:",avg_metric)
    return avg_metric

In [21]:
org_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images"

In [29]:
org_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images"

#AVIF Direct
recon_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images\AVIF_C"
df=batch_metrics(org_path,recon_path)
avg_metric=average_batch_metrics(df,"AVIF_summary.txt")

Images in dataset: 50
First filename: 000000_left.png
Images in dataset: 50
First filename: 000000_left_c.avif
Doing for: 000000_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000000_left.png
Doing for: 000001_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000001_left.png
Doing for: 000002_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000002_left.png
Doing for: 000003_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000003_left.png
Doing for: 000004_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000004_left.png
Doing for: 000005_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000005_left.png
Doing for: 000006_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000006_left.png
Doing for: 000007_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000007_left.png
Doing for: 000008_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\ima

In [30]:

write_dct_file(avg_metric,"AVIF_METRIC_AVG.txt")

In [31]:
#WEBP Direct
recon_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images\WEBP_C"
df_w=batch_metrics(org_path,recon_path)
avg_metric_w=average_batch_metrics(df_w,"WEBP_summary.txt")

Images in dataset: 50
First filename: 000000_left.png
Images in dataset: 50
First filename: 000000_left_c.webp
Doing for: 000000_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000000_left.png
Doing for: 000001_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000001_left.png
Doing for: 000002_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000002_left.png
Doing for: 000003_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000003_left.png
Doing for: 000004_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000004_left.png
Doing for: 000005_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000005_left.png
Doing for: 000006_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000006_left.png
Doing for: 000007_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000007_left.png
Doing for: 000008_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\ima

In [33]:
#JPEG Direct
recon_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images\JPG_C"
df_j=batch_metrics(org_path,recon_path)
avg_metric_j=average_batch_metrics(df_j)

Images in dataset: 50
First filename: 000000_left.png
Images in dataset: 50
First filename: 000000_left_c.jpg
Doing for: 000000_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000000_left.png
Doing for: 000001_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000001_left.png
Doing for: 000002_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000002_left.png
Doing for: 000003_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000003_left.png
Doing for: 000004_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000004_left.png
Doing for: 000005_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000005_left.png
Doing for: 000006_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000006_left.png
Doing for: 000007_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000007_left.png
Doing for: 000008_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\imag

In [39]:
#DL Based
recon_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images\Data_webp"
df_dl=batch_metrics(org_path,recon_path)
avg_metric_dl=average_batch_metrics(df_dl,"DL_summary.txt")

Images in dataset: 50
First filename: 000000_left.png
Images in dataset: 50
First filename: 000000_left_c.webp
Doing for: 000000_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000000_left.png
Doing for: 000001_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000001_left.png
Doing for: 000002_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000002_left.png
Doing for: 000003_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000003_left.png
Doing for: 000004_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000004_left.png
Doing for: 000005_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000005_left.png
Doing for: 000006_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000006_left.png
Doing for: 000007_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\images\000007_left.png
Doing for: 000008_left.png
C:\Swapnil\Narrowband_DRONE\compression_experiment\ima

In [73]:
#DL Based AVIF
recon_path=r"C:\Swapnil\Narrowband_DRONE\compression_experiment\images\Data_avif"
#df_dl_av=batch_metrics(org_path,recon_path)
avg_metric_dl_av=average_batch_metrics(df_dl_av)

average_metrics: {'Dataset_size': 50, 'SSIM': 0.81, 'PSNR': 27.44, 'LPIPS': 0.17, 'CR': 135.22, 'ORG_SIZE': 10101.73, 'RECON_SIZE': 83.16, 'ORG_SIFT': 61418.66, 'RECON_SIFT': 54629.64, 'SIFT_PER_RED': 16.33}
